In [1]:
import os
import glob
from PIL import Image
from google.colab import drive
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers, models
from ipywidgets import Button, HBox, VBox, Output, Layout
from IPython.display import display, clear_output
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Find Image size

In [2]:
image_folder= '/content/drive/My Drive/FYP/Cusp Images - SEM'
image_files = sorted(os.listdir(image_folder))
image_path = os.path.join(image_folder, image_files[0])

# Read the grayscale image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

cropped_image = image[:int(image.shape[0] * 0.93), :]

print(f"Image shape: {image.shape}")  # (Height, Width)
print(f"Cropped Image shape: {cropped_image.shape}")  # (Height, Width)

Image shape: (1920, 2560)
Cropped Image shape: (1785, 2560)


In [3]:

import keras

model_CNN = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(512, 512, 1)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Conv2D(256, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')  # Adjust if needed
])

model_CNN.summary()
# Compile the model
model_CNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])





/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 510, 510, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 255, 255, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 253, 253, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 126, 126, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 124, 124, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 62, 62, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 60, 60, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 30, 30, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 230400)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      58,982,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 59,404,682 (226.61 MB)

 Trainable params: 59,404,682 (226.61 MB)

 Non-trainable params: 0 (0.00 B)

: 

In [42]:
import matplotlib.pyplot as plt
import tensorflow as tf
image_path = "/path/to/image.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

def compute_saliency_map(model, image):
    """
    Compute the saliency map for a given image and model.

    Args:
    - model: Trained Keras model
    - image: Grayscale input image (should have shape (28, 28, 1))

    Returns:
    - Saliency map as a NumPy array
    """





    with tf.GradientTape() as tape:
        tape.watch(image_tensor)
        prediction = model(image_tensor)  # Get prediction

    grads = tape.gradient(prediction, image_tensor)  # Compute gradients
    # saliency = (saliency - np.min(saliency)) / (np.max(saliency) - np.min(saliency))
    saliency = tf.reduce_max(tf.abs(grads), axis=-1)  # Maximum absolute gradient for saliency map
    saliency = tf.squeeze(saliency)  # Remove dimensions of size 1

    return saliency.numpy()

In [43]:


# Get list of image filenames
image_files = sorted(os.listdir(image_folder))[:2]  # Process only the first 20 images

for filename in tqdm(image_files, desc="Processing Images"):
    image_path = os.path.join(image_folder, filename)

    # Read the grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


    # Crop the bottom 7% (removing tag strip)
    cropped_image = image[:int(image.shape[0] * 0.93), :]

    # Resize to (512, 512) for the model
    cropped_image = cv2.resize(cropped_image, (512, 512))

    # Convert to Tensor
    image_tensor = tf.convert_to_tensor(cropped_image, dtype=tf.float32)

    # Add batch and channel dimensions (make it (1, 512, 512, 1))
    image_tensor = tf.expand_dims(image_tensor, axis=0)  # Add batch dimension
    image_tensor = tf.expand_dims(image_tensor, axis=-1)


    # Compute the saliency map
    saliency = compute_saliency_map(model_CNN, image_tensor)

    # Plot the original image and saliency map
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    ax[0].imshow(cropped_image, cmap="gray")  # Corrected: No unnecessary reshape
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    ax[1].imshow(saliency, cmap="jet")  # Saliency Map
    ax[1].set_title("Saliency Map")
    ax[1].axis("off")

    plt.show()

Processing Images:   0%|          | 0/2 [00:01<?, ?it/s]


UnboundLocalError: cannot access local variable 'saliency' where it is not associated with a value